# 03. Summary Memory + Tool Calling 통합 학습

이 노트북에서는 **Summary Memory** 방식과 **Tool Calling**을 LangGraph + SQLiteSaver로 통합 구현합니다.

## 학습 목표
- Summary Memory의 개념과 특징 이해
- 대화 내용을 텍스트로 요약하는 기법
- Window Memory + Summary 하이브리드 관리
- SQLiteSaver를 사용한 영구 메모리 저장

---

## Buffer Memory vs Window Memory vs Summary Memory 비교

| 구분 | Buffer Memory | Window Memory | **Summary Memory** |
|------|---------------|---------------|--------------------|
| **저장 방식** | 모든 메시지 누적 | 최근 K개만 유지 | **최근 N개 + 요약 텍스트** |
| **토큰 비용** | 대화 길이에 비례 증가 | 일정하게 유지 | **낮음 (요약으로 압축)** |
| **장점** | 전체 맥락 보존 | 비용 효율적 | **비용 효율 + 전체 맥락 유지** |
| **단점** | 비용 증가, 컨텍스트 초과 위험 | 오래된 정보 손실 | **요약 시 추가 LLM 호출** |
| **적합한 상황** | 짧은 대화, 맥락이 중요한 작업 | 긴 대화, 비용 관리 필요 시 | **장기 대화, 맥락 중요** |

### Summary Memory의 핵심 아이디어

```
초기 상태 (메시지 8개, 대화 4쌍):
┌─────────────────────────────────────────────┐
│ 1. User: 안녕                                │ ← 대화 1쌍
│ 2. AI: 안녕하세요                            │
│ 3. User: 내 이름은 영희야                     │ ← 대화 2쌍
│ 4. AI: 반갑습니다 영희님                      │
│ 5. User: 나는 25살이야                       │ ← 대화 3쌍
│ 6. AI: 알겠습니다                            │
│ 7. User: 서울에 살아                         │ ← 대화 4쌍 (3쌍 초과!)
│ 8. AI: 네, 서울 거주시는군요                  │
└─────────────────────────────────────────────┘
              ↓ (3쌍 초과, 가장 오래된 2쌍 요약)

처리 후:
┌─────────────────────────────────────────────┐
│ Summary: "사용자 영희가 인사하고 자기소개를   │
│          시작했습니다."                       │
│ ─────────────────────────────────────────── │
│ 5. User: 나는 25살이야                       │ ← 최근 3쌍 유지
│ 6. AI: 알겠습니다                            │
│ 7. User: 서울에 살아                         │
│ 8. AI: 네, 서울 거주시는군요                  │
└─────────────────────────────────────────────┘
```

### 핵심 파라미터

- **WINDOW_SIZE = 6**: 최근 6개 메시지 유지 (대화 3쌍)
- **SUMMARY_WINDOW = 2**: 가장 오래된 2쌍 요약 대상

### 중요: Window 카운트 규칙

**카운트 대상 (Window 크기 계산):**
- ✅ HumanMessage (User 메시지)
- ✅ AIMessage with content (tool_calls 없는 실제 응답)

**카운트 제외 (항상 유지):**
- ❌ AIMessage with tool_calls (도구 호출)
- ❌ ToolMessage (도구 실행 결과)

**예시:**
```
1. User: "25+37은?"                    ← 카운트 1
2. AI: [tool_calls: calculator]       ← 카운트 제외 (유지)
3. Tool: "62"                         ← 카운트 제외 (유지)
4. AI: "62입니다"                     ← 카운트 2
5. User: "내 이름은 영희"             ← 카운트 3
6. AI: "반갑습니다"                   ← 카운트 4
```

### 그래프 구조 차이

```
Buffer Memory:                    Window Memory:
┌───────┐                         ┌───────┐
│ START │                         │ START │
└───┬───┘                         └───┬───┘
    ▼                                 ▼
┌─────────┐                       ┌─────────┐
│ chatbot │◄─┐                    │ chatbot │◄─┐
└────┬────┘  │                    └────┬────┘  │
     │       │                         │       │
  ┌──┴──┐    │                      ┌──┴──┐    │
  ▼     ▼    │                      ▼     ▼    │
tools  END   │                    tools  window_memory
  │          │                      │         │
  └──────────┘                      │         ▼
                                    │       END
                                    └─────────┘

Summary Memory:
┌───────┐
│ START │
└───┬───┘
    ▼
┌─────────┐
│ chatbot │◄─┐
└────┬────┘  │
     │       │
  ┌──┴──┐    │
  ▼     ▼    │
tools  summarizer
  │         │
  │         ▼
  │       END
  └─────────┘
```

---

## 1. 패키지 설치

In [1]:
!pip install langchain langchain-anthropic langgraph langgraph-checkpoint-sqlite -q
!pip install langchain-community wikipedia duckduckgo-search ddgs -q
print("설치 완료!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.2/388.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


---

## 2. 모델 선택

In [2]:
# ===== Claude 4.5 모델 선택 =====

# 1. Haiku 4.5 - 가장 빠르고 저렴 (테스트용 추천)
MODEL = "claude-haiku-4-5-20251001"

# 2. Sonnet 4.5 - 균형 잡힌 성능
# MODEL = "claude-sonnet-4-5-20250929"

# 3. Opus 4.5 - 최고 성능
# MODEL = "claude-opus-4-5-20251101"

print(f"선택된 모델: {MODEL}")

선택된 모델: claude-haiku-4-5-20251001


---

## 3. API 키 설정

In [3]:
import os

# 방법 1: 직접 입력
# os.environ["ANTHROPIC_API_KEY"] = "sk-ant-여기에-API-키-입력"

# 방법 2: Colab Secrets (권장)
try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
    print("Colab Secrets에서 API 키를 불러왔습니다!")
except:
    if "ANTHROPIC_API_KEY" in os.environ:
        print("환경 변수에서 API 키를 찾았습니다!")
    else:
        print("API 키를 직접 입력해주세요.")

Colab Secrets에서 API 키를 불러왔습니다!


---

## 4. LLM 초기화

In [4]:
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage, RemoveMessage

# 메인 LLM (대화용)
llm = ChatAnthropic(
    model=MODEL,
    max_tokens=1024
)

# 요약용 LLM (더 저렴한 모델 사용)
llm_summarizer = ChatAnthropic(
    model="claude-haiku-4-5-20251001",
    max_tokens=500
)

print("LLM 초기화 완료!")
print(f"  - 대화용: {MODEL}")
print(f"  - 요약용: claude-haiku-4-5-20251001")

LLM 초기화 완료!
  - 대화용: claude-haiku-4-5-20251001
  - 요약용: claude-haiku-4-5-20251001


---

## 5. 커스텀 도구 정의

In [5]:
from langchain_core.tools import tool
from datetime import datetime
import random

@tool
def get_current_time() -> str:
    """현재 날짜와 시간을 조회합니다. 사용자가 시간이나 날짜를 물어볼 때 사용하세요."""
    now = datetime.now()
    return f"현재 시간: {now.strftime('%Y년 %m월 %d일 %H시 %M분')}"

@tool
def calculator(a: float, b: float, operation: str) -> str:
    """두 숫자의 사칙연산을 수행합니다.

    Args:
        a: 첫 번째 숫자
        b: 두 번째 숫자
        operation: 수행할 연산 (add, subtract, multiply, divide)
    """
    operations = {
        "add": lambda x, y: x + y,
        "subtract": lambda x, y: x - y,
        "multiply": lambda x, y: x * y,
        "divide": lambda x, y: x / y if y != 0 else "오류: 0으로 나눌 수 없습니다"
    }

    if operation not in operations:
        return f"지원하지 않는 연산: {operation}"

    result = operations[operation](a, b)
    op_symbols = {"add": "+", "subtract": "-", "multiply": "×", "divide": "÷"}
    return f"{a} {op_symbols[operation]} {b} = {result}"

@tool
def random_number(min_val: int, max_val: int) -> str:
    """지정된 범위에서 랜덤 숫자를 생성합니다.

    Args:
        min_val: 최소값
        max_val: 최대값
    """
    num = random.randint(min_val, max_val)
    return f"{min_val}부터 {max_val} 사이의 랜덤 숫자: {num}"

custom_tools = [get_current_time, calculator, random_number]
print(f"커스텀 도구 {len(custom_tools)}개 정의 완료!")

커스텀 도구 3개 정의 완료!


---

## 6. 공식 외부 도구 설정

In [6]:
from langchain_community.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain_community.utilities import WikipediaAPIWrapper

# Wikipedia 도구
wikipedia = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(
        lang="ko",
        top_k_results=1,
        doc_content_chars_max=500
    )
)

# DuckDuckGo 검색 도구
duckduckgo = DuckDuckGoSearchRun()

external_tools = [wikipedia, duckduckgo]
print(f"외부 도구 {len(external_tools)}개 설정 완료!")

외부 도구 2개 설정 완료!


---

## 7. 모든 도구 통합

In [7]:
# 모든 도구 통합
all_tools = custom_tools + external_tools

# LLM에 도구 바인딩
llm_with_tools = llm.bind_tools(all_tools)

print(f"총 {len(all_tools)}개 도구가 LLM에 바인딩되었습니다:")
for i, t in enumerate(all_tools, 1):
    print(f"  {i}. {t.name}")

총 5개 도구가 LLM에 바인딩되었습니다:
  1. get_current_time
  2. calculator
  3. random_number
  4. wikipedia
  5. duckduckgo_search


---

## 8. SQLite 체크포인터 설정

In [8]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

DB_PATH = "summary_memory.db"

# SQLite 연결
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
memory = SqliteSaver(conn)

print(f"SQLite 메모리가 '{DB_PATH}' 파일에 설정되었습니다.")

SQLite 메모리가 'summary_memory.db' 파일에 설정되었습니다.


---

## 9. 📝 Summary Memory 핵심 로직

### 작동 방식

1. **Window 카운트**: HumanMessage + AIMessage(content만) 기준
2. **카운트 제외**: AIMessage(tool_calls), ToolMessage → 항상 유지
3. **Window 초과 시**: 가장 오래된 2쌍(4개) 요약
4. **기존 Summary 있으면**: 기존 Summary + 2쌍 → 새 Summary
5. **기존 Summary 없으면**: 2쌍만 → Summary

### Buffer/Window와의 차이점

| 구분 | Buffer | Window | **Summary** |
|------|--------|--------|-------------|
| 메시지 처리 | 무한 누적 | RemoveMessage로 삭제 | **RemoveMessage + 요약** |
| 토큰 비용 | 증가 | 고정 | **낮음 (요약 압축)** |
| 정보 손실 | 없음 | 높음 | **낮음 (요약 형태)** |
| 추가 비용 | 없음 | 없음 | **요약 시 LLM 호출** |

In [9]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages

# Summary Memory 설정
WINDOW_SIZE = 6  # 최근 6개 메시지 유지 (대화 3쌍)
SUMMARY_WINDOW = 2  # 가장 오래된 2쌍 요약

class State(TypedDict):
    messages: Annotated[list, add_messages]
    summary: str  # 요약 텍스트

def count_conversation_messages(messages: list) -> list:
    """Window 카운트 대상 메시지만 필터링

    카운트 대상:
    - HumanMessage (User)
    - AIMessage with content (tool_calls 없는 실제 응답)

    카운트 제외 (항상 유지):
    - AIMessage with tool_calls
    - ToolMessage
    """
    counted = []
    for msg in messages:
        if isinstance(msg, HumanMessage):
            counted.append(msg)
        elif isinstance(msg, AIMessage):
            # tool_calls 없고 content 있는 것만 카운트
            if not hasattr(msg, 'tool_calls') or not msg.tool_calls:
                if hasattr(msg, 'content') and msg.content:
                    counted.append(msg)
    return counted

def create_summary(messages_to_summarize: list, existing_summary: str) -> str:
    """메시지들을 요약

    Args:
        messages_to_summarize: 요약할 메시지 리스트
        existing_summary: 기존 요약 텍스트 (있으면 포함)

    Returns:
        새로운 요약 텍스트
    """
    # 대화 내용을 텍스트로 변환
    conversation_text = "\n".join(
        f"{type(msg).__name__}: {msg.content}"
        for msg in messages_to_summarize
        if hasattr(msg, 'content')
    )

    # 요약 프롬프트
    if existing_summary:
        prompt = f"""기존 요약:
{existing_summary}

추가 대화:
{conversation_text}

위 기존 요약과 추가 대화를 합쳐서 간결하게 요약하세요. 중요한 정보만 포함하세요."""
    else:
        prompt = f"""다음 대화를 간결하게 요약하세요. 중요한 정보만 포함하세요.

대화:
{conversation_text}"""

    try:
        response = llm_summarizer.invoke([HumanMessage(content=prompt)])
        return response.content.strip()
    except Exception as e:
        print(f"⚠️ 요약 생성 실패: {e}")
        return existing_summary if existing_summary else "요약 생성 실패"

print("=" * 60)
print("✅ Summary Memory 로직 정의 완료!")
print("=" * 60)
print(f"  - Window Size: {WINDOW_SIZE}개 (대화 {WINDOW_SIZE // 2}쌍)")
print(f"  - Summary Window: {SUMMARY_WINDOW}쌍 (오래된 {SUMMARY_WINDOW * 2}개 메시지)")
print(f"\n카운트 대상:")
print(f"  ✅ HumanMessage")
print(f"  ✅ AIMessage (content만, tool_calls 제외)")
print(f"\n카운트 제외 (항상 유지):")
print(f"  ❌ AIMessage with tool_calls")
print(f"  ❌ ToolMessage")

✅ Summary Memory 로직 정의 완료!
  - Window Size: 6개 (대화 3쌍)
  - Summary Window: 2쌍 (오래된 4개 메시지)

카운트 대상:
  ✅ HumanMessage
  ✅ AIMessage (content만, tool_calls 제외)

카운트 제외 (항상 유지):
  ❌ AIMessage with tool_calls
  ❌ ToolMessage


---

## 10. 그래프 노드 정의

### 노드 구조 (Buffer/Window와 비교)

| 노드 | Buffer | Window | **Summary** |
|------|--------|--------|-------------|
| chatbot | ✅ | ✅ | ✅ |
| tools | ✅ | ✅ | ✅ |
| window_memory | ❌ | ✅ | ❌ |
| **summarizer** | ❌ | ❌ | **✅ (신규)** |

In [11]:
from langgraph.prebuilt import ToolNode
from langchain_core.runnables import RunnableConfig

# 1. Chatbot 노드 (공통)
def chatbot(state: State, config: RunnableConfig):
    """메인 챗봇 노드"""
    messages = state["messages"]
    summary = state.get("summary", "")

    # 요약이 있으면 시스템 메시지로 추가
    if summary:
        system_msg = SystemMessage(
            content=f"""이전 대화 요약:
{summary}

위 요약을 참고하여 사용자와 대화하세요."""
        )
        messages_with_context = [system_msg] + messages
    else:
        messages_with_context = messages

    response = llm_with_tools.invoke(messages_with_context, config)
    return {"messages": [response]}

# 2. Tools 노드 (공통)
tool_node = ToolNode(all_tools)

# 3. Summarizer 노드 (신규)
def summarizer(state: State) -> dict:
    """📝 Summary Memory 핵심 노드

    작동 로직:
    1. 카운트 대상 메시지만 추출 (HumanMessage + AIMessage content)
    2. 카운트 대상이 6개 초과하면 가장 오래된 2쌍(4개) 요약
    3. 기존 summary 있으면 기존 + 2쌍 요약, 없으면 2쌍만 요약
    4. 요약된 메시지 삭제 (tool_calls, ToolMessage는 자동 유지)
    """
    messages = state["messages"]
    summary = state.get("summary", "")

    # 카운트 대상 메시지만 추출
    counted_msgs = count_conversation_messages(messages)

    # Window Size 이하면 그대로 유지
    if len(counted_msgs) <= WINDOW_SIZE:
        return {}

    # 가장 오래된 2쌍(4개) 요약 대상
    to_summarize = counted_msgs[:SUMMARY_WINDOW * 2]

    print(f"\n  📝 요약 시작: {len(to_summarize)}개 메시지")
    if summary:
        print(f"  📋 기존 요약 존재 (길이: {len(summary)}자)")

    # 새 요약 생성
    new_summary = create_summary(to_summarize, summary)

    print(f"  ✅ 새 요약 생성 완료 (길이: {len(new_summary)}자)")

    # 요약된 메시지 삭제 (RemoveMessage 사용)
    remove_messages = [RemoveMessage(id=m.id) for m in to_summarize]

    return {
        "messages": remove_messages,
        "summary": new_summary
    }

print("✅ 그래프 노드 정의 완료!")
print("  - chatbot: 메인 대화 처리")
print("  - tools: 도구 실행")
print("  - summarizer: 대화 요약 (신규)")

✅ 그래프 노드 정의 완료!
  - chatbot: 메인 대화 처리
  - tools: 도구 실행
  - summarizer: 대화 요약 (신규)


---

## 11. 라우터 함수 정의

In [12]:
def route_tools(state: State) -> str:
    """도구 사용 여부에 따라 라우팅"""
    last_message = state["messages"][-1]

    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"

    return "summarizer"

print("✅ 라우터 함수 정의 완료!")

✅ 라우터 함수 정의 완료!


---

## 12. LangGraph 구축

In [13]:
from langgraph.graph import StateGraph, START, END

# 그래프 생성
workflow = StateGraph(State)

# 노드 추가
workflow.add_node("chatbot", chatbot)
workflow.add_node("tools", tool_node)
workflow.add_node("summarizer", summarizer)

# 엣지 연결
workflow.add_edge(START, "chatbot")
workflow.add_conditional_edges(
    "chatbot",
    route_tools,
    {"tools": "tools", "summarizer": "summarizer"}
)
workflow.add_edge("tools", "chatbot")
workflow.add_edge("summarizer", END)

# 컴파일 (SQLite 체크포인터 포함)
graph = workflow.compile(checkpointer=memory)

print("=" * 60)
print("✅ Summary Memory 그래프 구축 완료!")
print("=" * 60)
print("\n그래프 구조:")
print("  START → chatbot → [tools or summarizer] → END")
print("            ↑            │")
print("            └────────────┘")

✅ Summary Memory 그래프 구축 완료!

그래프 구조:
  START → chatbot → [tools or summarizer] → END
            ↑            │
            └────────────┘


---

## 13. 채팅 함수 정의

In [14]:
def chat(user_message: str, session_id: str, verbose: bool = True) -> str:
    """사용자 메시지를 처리하고 응답 반환"""
    config = {"configurable": {"thread_id": session_id}}

    if verbose:
        print(f"\n{'=' * 60}")

    # 그래프 실행
    response = graph.invoke(
        {"messages": [HumanMessage(content=user_message)]},
        config=config
    )

    # 응답 추출
    if "messages" in response and response["messages"]:
        last_message = response["messages"][-1]
        if hasattr(last_message, "content"):
            return last_message.content

    return "응답을 생성할 수 없습니다."

def show_memory_status(session_id: str):
    """현재 메모리 상태 확인"""
    config = {"configurable": {"thread_id": session_id}}
    state = graph.get_state(config)

    messages = state.values.get("messages", [])
    summary = state.values.get("summary", "")

    # 카운트 대상 메시지
    counted_msgs = count_conversation_messages(messages)

    print("\n" + "=" * 60)
    print("🔍 Summary Memory 상태")
    print("=" * 60)

    # 요약 출력
    print(f"\n📝 요약:")
    if summary:
        print(f"  {summary}")
    else:
        print("  (없음)")

    # 최근 메시지 출력
    print(f"\n💬 최근 메시지 (전체: {len(messages)}개, 카운트 대상: {len(counted_msgs)}/{WINDOW_SIZE}개):")
    for i, msg in enumerate(messages, 1):
        if isinstance(msg, HumanMessage):
            print(f"  {i}. 👤 User: {msg.content[:50]}...")
        elif isinstance(msg, AIMessage):
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                print(f"  {i}. 🤖 AI: [도구 호출: {msg.tool_calls[0]['name']}] (카운트 제외)")
            else:
                content = msg.content if isinstance(msg.content, str) else str(msg.content)
                print(f"  {i}. 🤖 AI: {content[:50]}...")
        elif isinstance(msg, ToolMessage):
            print(f"  {i}. 🔧 Tool: {str(msg.content)[:50]}... (카운트 제외)")

    # 토큰 추정
    summary_chars = len(summary)
    msg_chars = sum(
        len(msg.content) for msg in messages
        if hasattr(msg, 'content') and isinstance(msg.content, str)
    )
    total_chars = summary_chars + msg_chars

    print(f"\n📊 토큰 추정:")
    print(f"  - 요약: ~{summary_chars // 4} 토큰")
    print(f"  - 최근 메시지: ~{msg_chars // 4} 토큰")
    print(f"  - 총합: ~{total_chars // 4} 토큰")
    print("=" * 60)

print("✅ 채팅 함수 정의 완료!")

✅ 채팅 함수 정의 완료!


---

## 14. 🧪 Summary Memory 작동 테스트

### 테스트 시나리오
1. 개인 정보 4쌍 입력 (이름, 나이, 거주지, 취미)
2. Window Size 초과 시 가장 오래된 2쌍 요약
3. 요약 후에도 정보 기억 여부 확인

In [15]:
print("=" * 60)
print("🧪 Summary Memory 작동 테스트")
print("=" * 60)

SESSION_ID = "summary-test"

conversations = [
    "안녕! 나는 영희야.",
    "나는 25살이야.",
    "서울에 살고 있어.",
    "취미는 독서야.",  # 대화 4쌍 (8개) - 3쌍 초과, 가장 오래된 2쌍 요약됨
    "내 이름이 뭐라고 했지?",  # 요약에서 검색 가능해야 함
]

for i, msg in enumerate(conversations, 1):
    print(f"\n{'=' * 60}")
    print(f"[{i}] 사용자: {msg}")
    response = chat(msg, SESSION_ID, verbose=False)
    print(f"    AI: {response}")

    # 4번째와 5번째 대화 후 메모리 상태 확인
    if i in [4, 5]:
        show_memory_status(SESSION_ID)

🧪 Summary Memory 작동 테스트

[1] 사용자: 안녕! 나는 영희야.
    AI: 안녕하세요, 영희님! 만나서 반갑습니다! 😊

저는 여러분을 도와드릴 수 있는 AI 어시스턴트입니다. 계산, 시간 확인, 정보 검색 등 다양한 일을 도와드릴 수 있어요.

오늘 뭘 도와드릴까요?

[2] 사용자: 나는 25살이야.
    AI: 알겠습니다! 영희님은 25살이시군요. 😊

혹시 뭘 도와드릴 일이 있으신가요? 계산이 필요하거나, 정보를 찾으시거나, 현재 시간을 알고 싶으시다면 언제든지 말씀해 주세요!

[3] 사용자: 서울에 살고 있어.
    AI: 좋습니다! 영희님은 25살이고 서울에 살고 계시는군요. 👍

서울은 정말 좋은 도시네요! 혹시 서울에 대해 알고 싶은 것이 있거나, 다른 도움이 필요하신 게 있으면 편하게 말씀해 주세요.

뭔가 도와드릴 수 있는 일이 있으신가요?

[4] 사용자: 취미는 독서야.

  📝 요약 시작: 4개 메시지
  ✅ 새 요약 생성 완료 (길이: 84자)
    AI: 오, 독서가 취미시군요! 정말 좋은 취미입니다. 📚

영희님은 어떤 종류의 책을 좋아하세요? 소설, 에세이, 자기계발서, 역사서 등 다양한 장르가 있는데요.

혹시 책에 대해 알고 싶거나, 추천받고 싶은 책이 있다면 말씀해 주세요. 정보를 찾아드릴 수 있습니다! 😊

🔍 Summary Memory 상태

📝 요약:
  **요약:**

영희(25살)가 AI 어시스턴트와 인사를 나눴습니다. AI는 계산, 정보 검색, 시간 확인 등의 도움을 제공할 수 있다고 안내했습니다.

💬 최근 메시지 (전체: 4개, 카운트 대상: 4/6개):
  1. 👤 User: 서울에 살고 있어....
  2. 🤖 AI: 좋습니다! 영희님은 25살이고 서울에 살고 계시는군요. 👍

서울은 정말 좋은 도시네요! ...
  3. 👤 User: 취미는 독서야....
  4. 🤖 AI: 오, 독서가 취미시군요! 정말 좋은 취미입니다. 📚

영희님은 어떤 종류의 책을 좋아하세요...

📊 토

---

## 15. 도구 + Summary Memory 통합 테스트

도구 사용 시에도 Summary가 정상 작동하는지 확인합니다.

**중요:** tool_calls와 ToolMessage는 카운트에서 제외되지만 메시지에는 유지됩니다.

In [16]:
print("\n" + "=" * 60)
print("🔧 도구 + Summary Memory 통합 테스트")
print("=" * 60)

SESSION_ID_TOOL = "tool-summary"

tool_conversations = [
    "내 이름은 민수야.",
    "25 더하기 37은?",  # 도구 사용 (카운트 제외)
    "나는 30살이야.",
    "서울에 살아.",
    "100 곱하기 5는?",  # 도구 사용 (카운트 제외)
    "내 이름과 나이가 뭐였지?",  # 요약에서 가져와야 함
]

for i, msg in enumerate(tool_conversations, 1):
    print(f"\n[{i}] 사용자: {msg}")
    response = chat(msg, SESSION_ID_TOOL, verbose=False)
    print(f"    AI: {response}")

# 최종 메모리 상태
show_memory_status(SESSION_ID_TOOL)


🔧 도구 + Summary Memory 통합 테스트

[1] 사용자: 내 이름은 민수야.
    AI: 안녕하세요, 민수님! 만나서 반갑습니다. 😊

저는 당신을 도울 수 있는 AI 어시스턴트입니다. 필요하신 것이 있으시면 언제든지 말씀해 주세요. 다음과 같은 도움을 드릴 수 있습니다:

- **계산**: 수학 계산을 도와드립니다
- **시간 정보**: 현재 날짜와 시간을 알려드립니다
- **임의의 숫자**: 특정 범위에서 무작위 숫자를 생성합니다
- **정보 검색**: 위키피디아에서 일반적인 정보를 찾아드립니다
- **최신 뉴스**: 현재 일어나고 있는 이슈나 뉴스를 검색합니다

어떤 도움이 필요하신가요?

[2] 사용자: 25 더하기 37은?
    AI: 25 더하기 37은 **62**입니다! 😊

[3] 사용자: 나는 30살이야.
    AI: 알겠습니다, 민수님! 30살이시군요. 😊

혹시 나이와 관련해서 도움이 필요하신 것이 있으신가요? 예를 들어:
- 나이 계산
- 특정 연도에 대한 정보
- 기타 질문

무엇을 도와드릴까요?

[4] 사용자: 서울에 살아.

  📝 요약 시작: 4개 메시지
  ✅ 새 요약 생성 완료 (길이: 92자)
    AI: 좋습니다, 민수님! 서울에 사시는군요. 😊

서울은 한국의 수도이고 역동적이고 활기찬 도시네요!

서울에 대해 궁금한 점이 있으시거나, 서울 관련 정보가 필요하신가요? 아니면 다른 도움이 필요하신 가요?

언제든지 말씀해 주세요!

[5] 사용자: 100 곱하기 5는?
    AI: 100 곱하기 5는 **500**입니다! 🎯

[6] 사용자: 내 이름과 나이가 뭐였지?

  📝 요약 시작: 4개 메시지
  📋 기존 요약 존재 (길이: 92자)
  ✅ 새 요약 생성 완료 (길이: 112자)
    AI: 네, 기억하고 있습니다! 😊

- **이름**: 민수
- **나이**: 30살

그리고 서울에 사시고 있다고 하셨죠!

🔍 Summary Memory 상태

📝 요약:
  **요약:**

- 사용자 이

---

## 16. Summary 누적 테스트

여러 번 요약이 발생할 때 기존 summary와 잘 병합되는지 확인합니다.

In [17]:
print("=" * 60)
print("📚 Summary 누적 테스트")
print("=" * 60)

SESSION_ID_CUMULATIVE = "cumulative-summary"

cumulative_conversations = [
    "내 이름은 철수야",
    "나는 28살이야",
    "서울 강남에 살아",
    "직업은 개발자야",  # 첫 번째 요약 발생 (1,2쌍 요약)
    "Python을 좋아해",
    "취미는 등산이야",  # 두 번째 요약 발생 (기존 요약 + 3,4쌍)
    "내 직업과 나이가 뭐였지?",
]

for i, msg in enumerate(cumulative_conversations, 1):
    print(f"\n[{i}] {msg}")
    response = chat(msg, SESSION_ID_CUMULATIVE, verbose=False)

    # 마지막 대화에서만 응답 출력
    if i == len(cumulative_conversations):
        print(f"응답: {response}")

# 최종 메모리 상태 확인
show_memory_status(SESSION_ID_CUMULATIVE)

📚 Summary 누적 테스트

[1] 내 이름은 철수야

[2] 나는 28살이야

[3] 서울 강남에 살아

[4] 직업은 개발자야

  📝 요약 시작: 4개 메시지
  ✅ 새 요약 생성 완료 (길이: 87자)

[5] Python을 좋아해

[6] 취미는 등산이야

  📝 요약 시작: 4개 메시지
  📋 기존 요약 존재 (길이: 87자)
  ✅ 새 요약 생성 완료 (길이: 123자)

[7] 내 직업과 나이가 뭐였지?
응답: 네, 철수님! 😊

**직업:** 개발자
**나이:** 28살

입니다!

Python을 좋아하시고 등산을 취미로 즐기시는 28살의 개발자 철수님이시죠! 혹시 다른 정보가 필요하시거나 궁금한 점이 있으세요?

🔍 Summary Memory 상태

📝 요약:
  **요약:**

- 사용자 이름: 철수
- 사용자 나이: 28살
- 거주지: 서울 강남
- 직업: 개발자
- AI가 제공 가능한 서비스: 계산, 시간 확인, 난수 생성, 정보 검색, 일반 질문 답변, 프로그래밍 관련 지원

💬 최근 메시지 (전체: 6개, 카운트 대상: 6/6개):
  1. 👤 User: Python을 좋아해...
  2. 🤖 AI: 좋습니다, 철수님! Python을 좋아하시는군요! 🐍

Python은 정말 훌륭한 언어예요...
  3. 👤 User: 취미는 등산이야...
  4. 🤖 AI: 좋습니다, 철수님! 등산이 취미이시군요! ⛰️

등산은 정말 훌륭한 취미예요! 신체 건강은...
  5. 👤 User: 내 직업과 나이가 뭐였지?...
  6. 🤖 AI: 네, 철수님! 😊

**직업:** 개발자
**나이:** 28살

입니다!

Python을 ...

📊 토큰 추정:
  - 요약: ~30 토큰
  - 최근 메시지: ~245 토큰
  - 총합: ~276 토큰


---

## 17. 메모리 방식 비교 분석

In [18]:
print("=" * 70)
print("📊 Buffer vs Window vs Summary Memory 토큰 비교")
print("=" * 70)

avg_tokens_per_msg = 100  # 평균 메시지당 토큰
avg_summary_tokens = 150  # 평균 요약당 토큰

print(f"\n가정:")
print(f"  - 평균 메시지당 {avg_tokens_per_msg} 토큰")
print(f"  - 평균 요약당 {avg_summary_tokens} 토큰")
print(f"  - Summary: 최근 6개 + 요약 텍스트")
print(f"  - Window: 최근 6개")

print("\n" + "-" * 70)
print(f"{'대화 쌍':<12} {'Buffer':<15} {'Window':<15} {'Summary':<15} {'절감률':<15}")
print("-" * 70)

for pairs in [3, 5, 10, 20, 50]:
    msgs = pairs * 2  # 대화 쌍당 2개 메시지

    # Buffer: 모든 메시지
    buffer_tokens = msgs * avg_tokens_per_msg

    # Window: 최근 6개만
    window_tokens = min(msgs, WINDOW_SIZE) * avg_tokens_per_msg

    # Summary: 최근 6개 + 요약
    recent_tokens = WINDOW_SIZE * avg_tokens_per_msg
    # 요약 횟수: (전체 쌍 - 3) / 2 (대략)
    summary_count = max(0, (pairs - 3) // 2)
    summary_tokens_total = recent_tokens + (summary_count * avg_summary_tokens)

    # 절감률 (Buffer 대비)
    savings = (1 - summary_tokens_total / buffer_tokens) * 100 if buffer_tokens > 0 else 0

    print(f"{pairs:<12} {buffer_tokens:>6,}      {window_tokens:>6,}      {summary_tokens_total:>6,}      {savings:>6.1f}%")

print("\n💡 분석:")
print("  - Buffer: 대화가 길어질수록 토큰 비용 급증")
print("  - Window: 토큰 고정, 오래된 정보 완전 손실")
print("  - Summary: 토큰 효율 + 요약으로 전체 맥락 유지")
print("=" * 70)

📊 Buffer vs Window vs Summary Memory 토큰 비교

가정:
  - 평균 메시지당 100 토큰
  - 평균 요약당 150 토큰
  - Summary: 최근 6개 + 요약 텍스트
  - Window: 최근 6개

----------------------------------------------------------------------
대화 쌍         Buffer          Window          Summary         절감률            
----------------------------------------------------------------------
3               600         600         600         0.0%
5             1,000         600         750        25.0%
10            2,000         600       1,050        47.5%
20            4,000         600       1,800        55.0%
50           10,000         600       4,050        59.5%

💡 분석:
  - Buffer: 대화가 길어질수록 토큰 비용 급증
  - Window: 토큰 고정, 오래된 정보 완전 손실
  - Summary: 토큰 효율 + 요약으로 전체 맥락 유지


---

## 18. 인터랙티브 챗봇

In [19]:
def interactive_chat():
    """인터랙티브 챗봇 실행"""
    print("=" * 60)
    print(f"📝 Summary Memory 인터랙티브 챗봇")
    print("명령어: 'quit'=종료, 'memory'=메모리 상태 확인")
    print("=" * 60)

    session_id = f"interactive-{datetime.now().strftime('%H%M%S')}"

    while True:
        user_input = input("\n사용자: ").strip()

        if user_input.lower() in ['quit', '종료', 'exit', 'q']:
            print("\n챗봇을 종료합니다.")
            break

        if user_input.lower() == 'memory':
            show_memory_status(session_id)
            continue

        if not user_input:
            continue

        response = chat(user_input, session_id)
        print(f"\nAI: {response}")

# 실행하려면 주석 해제
# interactive_chat()

print("✅ 인터랙티브 챗봇 준비 완료!")
print("   interactive_chat() 함수를 호출하면 시작됩니다.")

✅ 인터랙티브 챗봇 준비 완료!
   interactive_chat() 함수를 호출하면 시작됩니다.


---

## 19. Summary Memory 특징 정리

### 💎 핵심 장점

1. **전체 맥락 유지**: 요약으로 오래된 대화 내용 보존
2. **토큰 효율**: 압축된 요약으로 비용 절감
3. **확장 가능**: 무한히 긴 대화도 관리 가능
4. **유연성**: Window Size와 Summary Window 조정 가능

### ⚠️ 주의사항

1. **요약 품질**: LLM 성능에 의존
2. **추가 비용**: 요약 생성 시 LLM 호출 필요
3. **세부 정보 손실**: 요약 과정에서 일부 정보 누락 가능
4. **지연 시간**: 요약 생성에 시간 소요

### 📊 메모리 방식 종합 비교

| 방식 | 토큰 비용 | 정보 보존 | 맥락 유지 | 복잡도 | 추천 상황 |
|------|----------|----------|----------|--------|----------|
| **Buffer** | 높음 (증가) | 완전 | 완전 | 낮음 | 짧은 대화, 개발 |
| **Window** | 고정 | 최근만 | 최근만 | 낮음 | 비용 중시, 단순 대화 |
| **Summary** | **낮음** | **요약 형태** | **전체 (요약)** | 중간 | **장기 대화, 맥락 중요** |

### 🎯 Summary Memory 최적 활용 시나리오

1. **고객 지원**: 긴 상담 기록 요약
2. **교육 챗봇**: 학습 진행 상황 누적
3. **프로젝트 관리**: 회의 내용 요약
4. **개인 비서**: 장기간 대화 이력 관리

### ⚙️ 파라미터 튜닝 가이드

| WINDOW_SIZE | SUMMARY_WINDOW | 적합한 상황 |
|-------------|----------------|-------------|
| 6 | 2 | 일반적인 대화 (기본값) |
| 10 | 3 | 맥락이 중요한 작업 |
| 4 | 1 | 빠른 요약, 비용 절감 |
| 8 | 2 | 균형 잡힌 설정 |

---

## 20. 정리 및 다음 단계

In [20]:
# SQLite 연결 닫기
conn.close()
print("데이터베이스 연결이 닫혔습니다.")
print(f"대화 기록은 '{DB_PATH}' 파일에 저장되어 있습니다.")

데이터베이스 연결이 닫혔습니다.
대화 기록은 'summary_memory.db' 파일에 저장되어 있습니다.


### ✅ 학습 완료 체크리스트

- ✅ Summary Memory 개념 이해
- ✅ Window + Summary 하이브리드 구조 구현
- ✅ 카운트 규칙 (tool_calls, ToolMessage 제외)
- ✅ 요약 생성 및 누적 메커니즘
- ✅ 도구 통합 및 실전 테스트
- ✅ Buffer/Window와 성능 비교 분석

### 🚀 다음 단계

**실전 프로젝트에 적용하기:**
1. 요약 품질 개선 (프롬프트 엔지니어링)
2. 요약 전략 최적화 (중요도 기반)
3. 외부 저장소 연동 (벡터 DB)
4. 요약 검색 기능 추가

### 💡 프로덕션 배포 시 고려사항

1. **요약 품질 모니터링**: 주기적으로 요약 품질 검증
2. **파라미터 튜닝**: 도메인에 맞게 WINDOW_SIZE 조정
3. **비용 최적화**: Summarizer에 저렴한 모델 사용
4. **캐싱**: 동일 요약 재사용
5. **A/B 테스팅**: Summary vs Window 성능 비교

### 🎓 핵심 차이점 요약

**Buffer Memory (15_01):**
- 모든 메시지 누적 저장
- 노드: chatbot, tools만

**Window Memory (15_02):**
- 최근 N개만 유지
- 노드: chatbot, tools, window_memory
- RemoveMessage로 오래된 것 삭제

**Summary Memory (15_03):**
- 최근 N개 + 요약 텍스트
- 노드: chatbot, tools, summarizer
- RemoveMessage + 요약 생성
- tool_calls/ToolMessage는 카운트 제외